In [1]:
import pandas as pd
import os
import glob

In [2]:
fulltext_df = pd.read_csv('full_text_review.csv')
abstract_df = pd.read_csv('title_abstract_screening.csv')

In [3]:
import os
import glob

def get_pdf_files(directory_path):
    pdf_files = []
    for file_path in glob.glob(os.path.join(directory_path, '*.pdf')):
        # Get only the file name from the path
        pdf_files.append(os.path.basename(file_path))
    return pdf_files

current_directory = os.getcwd()
target_directory = os.path.join(os.path.dirname(current_directory), 'data')

In [20]:
pdf_files_list = [os.path.splitext(file_name.replace(':', '/'))[0] for file_name in  get_pdf_files(target_directory)]
fulltext_DOI = list(fulltext_df['DOI'])
abstract_DOI = list(abstract_df['DOI'])

In [21]:
print(len(fulltext_DOI),
len(abstract_DOI))

184 2535


In [22]:
# Convert the lists to sets for faster set operations
pdf_files_set = set(pdf_files_list)
fulltext_set = set(fulltext_DOI)
abstract_set = set(abstract_DOI)

# Find common elements between pdf_files_set and fulltext_set
common_elements_ft = pdf_files_set.intersection(fulltext_set)

# Remove the common elements from fulltext_DOI and pdf_files_list
fulltext_DOI = [doi for doi in fulltext_DOI if doi not in common_elements_ft]

print("Modified fulltext_DOI:", len(fulltext_DOI))
print("Common elements:", len(common_elements_ft))

# Find common elements between pdf_files_set and abstract_set
common_elements_ab = pdf_files_set.intersection(abstract_set)

# Remove the common elements from abstract_DOI and pdf_files_list
abstract_DOI = [doi for doi in abstract_DOI if doi not in common_elements_ab]

print("\nModified abstract_DOI:", len(abstract_DOI))
print("Common elements:", len(common_elements_ab))


Modified fulltext_DOI: 178
Common elements: 6

Modified abstract_DOI: 2367
Common elements: 163


In [18]:
common_elements_ab

set()